# Castena - Chatbot for multilingual podcasts

* Document retrieval via langchain

In [ ]:
!pip install langchain huggingface_hub tiktoken -q
!pip install chromadb -q
!pip install PyPDF2 pypdf sentence_transformers -q
!pip install -U together -q
!pip install -U FlagEmbedding -q
!pip install googletrans==3.1.0a0 -q
!pip install spacy
!python -m spacy download es_core_news_lg
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.6 MB/s eta 

In [ ]:
!pip freeze > requirements.txt
!pip list --format=freeze > requirements.txt

In [1]:
# -- Import libraries
from   typing                      import Any, Dict, List, Mapping, Optional
from   pydantic                    import Extra, Field, root_validator
from   langchain.callbacks.manager import CallbackManagerForLLMRun
from   langchain.memory            import ConversationBufferWindowMemory
from   langchain.llms.base         import LLM
from   langchain.llms.utils        import enforce_stop_tokens
from   langchain.chains.llm        import LLMChain
from   langchain.utils             import get_from_dict_or_env
from   googletrans                 import Translator
from   langchain.vectorstores      import Chroma, FAISS
from   langchain.text_splitter     import RecursiveCharacterTextSplitter, CharacterTextSplitter
from   langchain.chains            import RetrievalQA, ReduceDocumentsChain, MapReduceDocumentsChain
from   langchain.evaluation.qa     import QAEvalChain
from   langchain.document_loaders  import TextLoader, DirectoryLoader
from   langchain.embeddings        import HuggingFaceEmbeddings
from   langchain.prompts           import PromptTemplate
from   langchain.schema            import prompt
from   langchain.chains.mapreduce  import MapReduceChain
from   langchain.chains.combine_documents.stuff import StuffDocumentsChain
from   sentence_transformers       import SentenceTransformer, util
from   tqdm                        import tqdm
import pandas                      as pd
import collections
import logging
import together
import textwrap
import spacy
import torch
import json
import os
import re

tqdm.pandas()

# -- Constants
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

#with open("./main/prompts/default_system_prompt.txt", "r") as f:
#  DEFAULT_SYSTEM_PROMPT = f.readlines()

with open("./Castena-main/prompts/default_system_prompt.txt", "r") as f:
  DEFAULT_SYSTEM_PROMPT = f.read()

# Setup API Key

In [2]:
os.environ["TOGETHER_API_KEY"] = "6101599d6e33e3bda336b8d007ca22e35a64c72cfd52c2d8197f663389fc50c5"

# Pre-process data

In [3]:
# -- Define auxiliar functions to clean/process data
translator = Translator()

# Carga el modelo de SpaCy para el idioma en el que está el texto de origen
nlp = spacy.load('es_core_news_lg')

def translate_text(text, target_lang='en'):
    # Traducir el texto sin los nombres propios
    translator = Translator()
    # Tokenizar el texto y encontrar nombres propios
    doc = nlp(text)
    named_entities = [ent.text for ent in doc if ent.pos_ == 'PROPN' and ent.dep_ in ['NNP', 'NN']]
    named_entities_list = []
    # Reemplazar los nombres propios con marcadores temporales
    for entity in named_entities:
        text = text.replace(entity, f'__{entity}__')
        named_entities_list.append(entity)

    translated_text = translator.translate(text, dest=target_lang).text
    final_translated_text = []

    i = 0

    for text in translated_text.split(' '):
      if '__' in text and len(named_entities_list):
        final_translated_text.append(named_entities_list[i])
        i+=1
      else:
        final_translated_text.append(text)
    return ' '.join(final_translated_text)

def capitalize_proper_nouns(text, nlp):
    # Correct punctuation signs
    text = text.replace(" .. ", "...")\
               .replace(" , ", ",")\
               .replace(" .... ", "...")\
               .replace(" ... ", "...")\
               .replace(",¿", " ¿")\
               .replace(",.", ",")\
               .replace("?.", "? ")

    # Analizar el texto con spaCy
    doc = nlp(text)

    # Inicializar una lista para almacenar el texto modificado
    modified_text = []

    # Recorrer el texto y capitalizar la primera letra de cada nombre propio
    for token in doc:
        if token.pos_ != "PROPN":
            # Si es un nombre propio y ya está en mayúscula, conservarlo
            modified_text.append(token.text)
        else:
            # De lo contrario, capitalizar la primera letra
            modified_text.append(token.text.capitalize())

    # Unir las palabras nuevamente en un solo texto
    modified_text = " ".join(modified_text)
    return modified_text

In [4]:
# -- Chunk to translate spanish transcripts if necessary
transcription_df = pd.read_table('../Downloads/worldcast_roberto_vaquero_transcription (1).txt', sep='|', header=None)
transcription_df.rename(columns={0: 'time', 1: 'speaker', 2: 'transcript'}, inplace=True)

transcription_df['time'] = pd.to_timedelta(transcription_df['time'])
transcription_df['speaker_change'] = transcription_df['speaker'] != transcription_df['speaker'].shift()

result = transcription_df.groupby(['speaker', transcription_df['speaker_change'].cumsum()]).agg({\
                                                                                                 'time': ['min', 'max'],
                                                                                                 'transcript': lambda x: '.'.join(x)
                                                                                                })
result.columns = result.columns.droplevel()
result.columns = ['min_time', 'max_time', 'transcript']
result.reset_index(inplace=True)
result['min_time'] = result['min_time'].apply(lambda x: str(x).replace('0 days ', ''))
result['max_time'] = result['max_time'].apply(lambda x: str(x).replace('0 days ', ''))

# -- Preprocess transcript
result['transcript'] = result['transcript'].apply(lambda x: capitalize_proper_nouns(x, nlp))

result['literal_transcript'] = 'Desde el instante ' + result['min_time'] + ' hasta ' + result['max_time'] + ' ' + result['speaker'] + ' dice: \"' + result['transcript'] + '\"'
result['literal_transcript'] = result['literal_transcript'].progress_apply(translate_text)
result = result.sort_values('min_time')
# Sample output to save as .txt file (uncomment following line)
# '\n\n'.join(result['literal_transcript'])

100%|██████████| 94/94 [00:53<00:00,  1.75it/s]


In [7]:
translated_transcription = '\n\n'.join(result['literal_transcript'])
with open('translated_worldcast_roberto_vaquero_transcription.txt', 'w') as f:
  f.write(translated_transcription)

# Setting up Together API


In [8]:
# Set your API key
together.api_key = os.environ["TOGETHER_API_KEY"]
# List available models and descriptons
models = together.Models.list()
# Set llama2 7b LLM
together.Models.start("togethercomputer/llama-2-7b-chat")

{'success': True,
 'value': 'bed483f828fbd80cce2c7cd213ebb6b338ee60af469580aa936987a86192969c-f96e767b62b60b0fa42b6e2b74e8ad012ea1fc87eeb38b292c4c9b79fb61afed',
 'wasAlreadyEnabled': True}

In [9]:
class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def clean_duplicates(self, transcription: str) -> str:
      lines = transcription.strip().split('\n')

      new_transcription = []

      for linea in lines:
          if linea.replace('CONTEXT:/n/n ', '').replace('/n', '') not in new_transcription and linea != '':
              new_transcription.append(linea.replace('CONTEXT:/n/n ', '').replace('/n', ''))
      # Create new transcription without duplicates
      new_transcription = '\n\n'.join(new_transcription).replace("""<</SYS>>
      """, """<</SYS>>
      CONTEXT: """)
      return new_transcription

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        cleaned_prompt   = self.clean_duplicates(prompt)
        output = together.Complete.create(cleaned_prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        cleaned_text = self.clean_duplicates(text)
        return cleaned_text


# LangChain multi-doc retriever with ChromaDB

***Key Points***
- Multiple Files - PDFs
- ChromaDB
- llama-2-7b-chat LLM
- BGE Embeddings (newest version)


## Setting up LangChain


## Load multiple and process documents

In [10]:
# Load and process the text files
loader = TextLoader('./translated_worldcast_roberto_vaquero_transcription.txt')
documents = loader.load()

In [11]:
len(documents)

1

In [12]:
# Splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

len(texts)

101

## Load HF BGE Embeddings

In [13]:
model_name = "BAAI/bge-base-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs=encode_kwargs
)

## create the DB

 T4 GPU

__Why Chroma instead of FAISS?__

Answer:

In terms of performance, there is no direct benchmark comparison available between **Chroma** and **FAISS**. This is because **FAISS** is not regularly used as a stand-alone vector database, so it is difficult to compare it directly with **Chroma**.

**FAISS** is designed for efficient similarity search, which can be crucial for applications involving large-scale semantic search. However, for a production environment, it may need to be built into a custom container or larger system to support CRUD operations, high availability, horizontal scalability, concurrent access, etc. It is built around an Index object. This object encapsulates the set of database vectors and optionally preprocesses them to make the search efficient. There are many types of indexes, but the simplest version performs a brute force Euclidean (L2) distance search.

On the other hand, **Chroma** is designed to run on your machine and was built to handle modern AI workloads, making it suitable for embedding-intensive applications.

Therefore, the choice between **Chroma** and **FAISS** depends on your specific use case. If you're looking for a standalone vector database that's easy to set up and use for local development, **Chroma** may be a good choice. If you need a tool for efficient similarity search and dense vector clustering and ready to build additional functionality around it, **FAISS** might be suitable.

In [14]:
%%time
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'
## Here is the nmew embeddings being used
embedding = model_norm

# -- Chroma DB
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

#vectordb = FAISS.from_documents(texts, embedding)

CPU times: user 3min 31s, sys: 32.9 s, total: 4min 4s
Wall time: 40.7 s


## Make a retriever

In [15]:
retriever = vectordb.as_retriever(search_type="similarity_score_threshold",
                                  search_kwargs={"k": 5, "score_threshold": 0.1})

## Make a chain

In [16]:
def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

In [17]:
sys_prompt = DEFAULT_SYSTEM_PROMPT
instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""
get_prompt(instruction, sys_prompt)

"[INST]<<SYS>>\nYou are an assistant. Your mission is to provide accurate answers to questions regarding the transcription of a YouTube interview.\n\nBe concise and omit disclaimers or default messages.\n\nDo not give your personal opinion or your personal conclusion.\n\nDo not make guesses or assumptions.\n\nIf you do not know the answer of the question, politely explain the issue.\n\nIf you do not have enough information to answer, please respond please don't share false information.\n\nIf the question has nothing to do with the transcript, answer the question without mentioning anything about the transcript.\n\nDo not add emojis to the response.\n<</SYS>>\n\nCONTEXT:/n/n {context}/n\n\nQuestion: {question}[/INST]"

In [18]:
llm = TogetherLLM(
    model= "togethercomputer/llama-2-7b-chat",
    temperature = 0.0,
    max_tokens = 1024
)

In [19]:
prompt_template = get_prompt(instruction, sys_prompt)

llama_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [20]:
# -- NOTE: in case we want to add memory, 'history' field must be added to the prompt
# HISTORY:/n/n {history}/n
#memory = ConversationBufferWindowMemory(k=1, memory_key="history", input_key="question")
#chain_type_kwargs = {"prompt": llama_prompt, "memory": memory}
chain_type_kwargs = {"prompt": llama_prompt}

In [21]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       retriever=retriever,
                                       chain_type_kwargs=chain_type_kwargs,
                                       return_source_documents=True)

In [22]:
## Cite sources
def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
  response = llm_response['result']
  return wrap_text_preserve_newlines(translate_text(response, target_lang='es'))

In [23]:
# Full example
query = "¿Cual es la opinion de Roberto Vaquero sobre el partido VOX? ¿En qué momento habla sobre VOX?"
translated_query = translate_text(query, target_lang='en')
llm_response = qa_chain(translated_query)
print(process_llm_response(llm_response))

La opinión de Roberto Vaquero sobre el partido VOX es que es un partido general similar a los partidos de
Estados Unidos. Menciona que están importando el sistema político de Estados Unidos y que no están enfocados
en un segmento específico de la población, sino en lo que se siente bien. También señala que en realidad no
defienden la democracia, la libertad de expresión, etc., sino que inmediatamente critican de forma crítica
cualquier cosa como discurso de odio peligroso.


In [ ]:
together.Models.stop("togethercomputer/llama-2-7b-chat")

{'success': True, 'wasAlreadyDisabled': True}

## Evaluation on custom dataset

In [24]:
with open('../Downloads/Castena-main/data/eval/eval_cleaned/worldcast_roberto_vaquero_eval_qa.json') as f:
    eval_dataset = json.load(f)

In [25]:
queries = [list(value.keys())[0] for value in eval_dataset]
outputs = [list(value.values())[0] for value in eval_dataset]

In [26]:
eval_gen_outputs = []

for query in tqdm(queries):
  translated_query = translate_text(query, target_lang='en')
  llm_response = qa_chain(str(translated_query))
  llm_response_translated = process_llm_response(llm_response)
  eval_gen_outputs.append(llm_response_translated)
print("Finished!")

100%|██████████| 28/28 [01:49<00:00,  3.90s/it]

Finished!


In [27]:
eval_gen_outputs_formatted = [{'query': translate_text(query), 'answer': translate_text(output), 'result': translate_text(answer)} for query, answer, output in tqdm(zip(queries, eval_gen_outputs, outputs))]
outputs_formatted          = [{'query': translate_text(query), 'answer': translate_text(answer)} for query, answer in tqdm(zip(queries, outputs))]

28it [00:49,  1.78s/it]
28it [00:31,  1.12s/it]


In [28]:
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(outputs_formatted, eval_gen_outputs_formatted)

In [29]:
# -- Update: 2023 - 10 - 23
correct_incorrect_outputs = [re.findall("(CORRECT|INCORRECT)", result['results'])[0] for result in graded_outputs]
counter = collections.Counter(correct_incorrect_outputs)
dict(counter)

{'CORRECT': 18, 'INCORRECT': 10}

In [30]:
# -- Update: 2023 - 10 - 20
correct_incorrect_outputs = [re.findall("(CORRECT|INCORRECT)", result['results'])[0] for result in graded_outputs]
counter = collections.Counter(correct_incorrect_outputs)
dict(counter)

{'CORRECT': 16, 'INCORRECT': 12}

In [ ]:
# -- Update: 2023 - 10 - 16
correct_incorrect_outputs = [re.findall("(CORRECT|INCORRECT)", result['results'])[0] for result in graded_outputs]
counter = collections.Counter(correct_incorrect_outputs)
dict(counter)

{'CORRECT': 15, 'INCORRECT': 13}

In [ ]:
# -- Update: 2023 - 10 - 15
correct_incorrect_outputs = [re.findall("(CORRECT|INCORRECT)", result['results'])[0] for result in graded_outputs]
counter = collections.Counter(correct_incorrect_outputs)
dict(counter)

{'CORRECT': 20, 'INCORRECT': 8}

In [ ]:
# -- Update: 2023 - 10 - 10
correct_incorrect_outputs = [re.findall("(CORRECT|INCORRECT)", result['results'])[0] for result in graded_outputs]
counter = collections.Counter(correct_incorrect_outputs)
dict(counter)

{'INCORRECT': 10, 'CORRECT': 18}

In [ ]:
correct_incorrect_outputs = [re.findall("(CORRECT|INCORRECT)", result['results'])[0] for result in graded_outputs]
counter = collections.Counter(correct_incorrect_outputs)
dict(counter)

{'CORRECT': 22, 'INCORRECT': 6}

### Sentence Similarity

In [30]:
# v1 -> all-mpnet-base-v2
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

similarity_scores_list = []
for original_output, gen_output in tqdm(zip(outputs, eval_gen_outputs)):
  query_embedding = model.encode(original_output)
  passage_embedding = model.encode([gen_output])
  similarity_scores_list.append(float(util.dot_score(query_embedding, passage_embedding)))

28it [00:15,  1.80it/s]


In [31]:
# -- Update: 2023 - 10 - 23
torch.mean(torch.tensor(similarity_scores_list)), torch.median(torch.tensor(similarity_scores_list))

(tensor(0.7110), tensor(0.7166))

In [33]:
# -- Update: 2023 - 10 - 20
torch.mean(torch.tensor(similarity_scores_list)), torch.median(torch.tensor(similarity_scores_list))

(tensor(0.7533), tensor(0.7742))

In [ ]:
# -- Update: 2023 - 10 - 16
torch.mean(torch.tensor(similarity_scores_list)), torch.median(torch.tensor(similarity_scores_list))

(tensor(0.7165), tensor(0.7622))

In [ ]:
# -- Update: 2023 - 10 - 15
torch.mean(torch.tensor(similarity_scores_list)), torch.median(torch.tensor(similarity_scores_list))

(tensor(0.6977), tensor(0.7144))

In [ ]:
# -- Update: 2023 - 10 - 10
torch.mean(torch.tensor(similarity_scores_list)), torch.median(torch.tensor(similarity_scores_list))

(tensor(0.7225), tensor(0.7406))

In [32]:
# v2 -> all-MiniLM-L6-v2
model = SentenceTransformer('all-MiniLM-L6-v2')

similarity_scores_list = []
for original_output, gen_output in tqdm(zip(outputs, eval_gen_outputs)):
  query_embedding = model.encode(original_output, convert_to_tensor=True)
  passage_embedding = model.encode(gen_output, convert_to_tensor=True)
  similarity_scores_list.append(float(util.pytorch_cos_sim(query_embedding, passage_embedding)))

28it [00:03,  9.08it/s]


In [33]:
# -- Update: 2023 - 10 - 23
torch.mean(torch.tensor(similarity_scores_list)), torch.median(torch.tensor(similarity_scores_list))

(tensor(0.7250), tensor(0.7354))

In [35]:
# -- Update: 2023 - 10 - 20
torch.mean(torch.tensor(similarity_scores_list)), torch.median(torch.tensor(similarity_scores_list))

(tensor(0.7368), tensor(0.7391))

In [ ]:
# -- Update: 2023 - 10 - 16
torch.mean(torch.tensor(similarity_scores_list)), torch.median(torch.tensor(similarity_scores_list))

(tensor(0.7231), tensor(0.7261))

### Using cosine similarity

In [34]:
cosine_similarity_list = []
for original_output, predicted in zip(outputs, eval_gen_outputs):
  original_output_emb = torch.tensor(model_norm.embed_documents([translate_text(original_output)]))
  predicted_emb       = torch.tensor(model_norm.embed_documents([translate_text(predicted)]))
  cosine_similarity   = torch.nn.functional.cosine_similarity(original_output_emb, predicted_emb, dim=1)
  cosine_similarity_list.append(cosine_similarity)

In [35]:
# -- Update: 2023 - 10 - 23
torch.mean(torch.tensor(cosine_similarity_list)), torch.median(torch.tensor(cosine_similarity_list))

(tensor(0.7856), tensor(0.8103))

In [37]:
# -- Update: 2023 - 10 - 20
torch.mean(torch.tensor(cosine_similarity_list)), torch.median(torch.tensor(cosine_similarity_list))

(tensor(0.7967), tensor(0.8307))

In [ ]:
# -- Update: 2023 - 10 - 16
torch.mean(torch.tensor(cosine_similarity_list)), torch.median(torch.tensor(cosine_similarity_list))

(tensor(0.7766), tensor(0.8047))

In [ ]:
# -- Update: 2023 - 10 - 15
torch.mean(torch.tensor(cosine_similarity_list)), torch.median(torch.tensor(cosine_similarity_list))

(tensor(0.7806), tensor(0.8041))

In [ ]:
# -- Update: 2023 - 10 - 10
torch.mean(torch.tensor(cosine_similarity_list)), torch.median(torch.tensor(cosine_similarity_list))

(tensor(0.7943), tensor(0.8198))

In [ ]:
torch.mean(torch.tensor(cosine_similarity_list)), torch.median(torch.tensor(cosine_similarity_list))

(tensor(0.7794), tensor(0.8230))

# Summary of transcription: map reduce technique

In [ ]:
with open("/content/translated_worldcast_roberto_vaquero_transcription.txt") as f:
    docs = f.read()

llm = TogetherLLM(
    model= "togethercomputer/llama-2-7b-chat",
    temperature = 0.0,
    max_tokens = 1024
)

# Map
with open("/content/map_template.txt", "r") as f:
  map_template = f.read()
map_prompt = PromptTemplate(template=map_template, input_variables=["docs"])
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
with open("/content/reduce_template.txt", "r") as f:
  reduce_template = f.read()
reduce_prompt = PromptTemplate(template=reduce_template, input_variables=["doc_summaries"])

# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    verbose=True,
    token_max=1024
)

# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
    verbose=True
)
text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 2000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = True,
)
split_docs = text_splitter.create_documents([docs])

In [ ]:
text_summary = map_reduce_chain.run(split_docs)
text_summary_translated = translate_text(text_summary, 'es')
print(text_summary_translated.replace(". ", ".\n"))



> Entering new MapReduceDocumentsChain chain...


Token indices sequence length is longer than the specified maximum sequence length for this model (5284 > 1024). Running this sequence through the model will result in indexing errors



> Finished chain.
Roberto Vaquero es un exmilitar y figura política que comparte su reflexión sobre la situación política y social actual en España y el mundo.
Expresa preocupación por la erosión de los valores tradicionales y la promoción de la ideología de género en la educación, la inmigración y otras áreas.
Vaquero cree que es importante defender el país y la cultura, pero también escuchar y considerar diferentes perspectivas.
Habla de sus experiencias personales, incluido su tiempo en prisión, y cómo esto ha dado forma a sus puntos de vista sobre la política y la vida.
Vaquero también habla del concepto de "corrección política" y su impacto en la sociedad, y de cómo cree que debe haber un mensaje y una alternativa a esta forma de pensar.
Vaquero enfatiza la importancia del desarrollo personal, el pensamiento crítico y valores como el honor y la palabra.
Destaca la necesidad de procesos más inclusivos y participativos para crear una nueva constitución en España que realmente repre

In [ ]:
together.Models.stop("togethercomputer/llama-2-7b-chat")

{'success': True}